**Downloading the fasttext Persian model**

In [ ]:
!pip install --upgrade --no-cache-dir gdown -q

In [ ]:
!gdown 1kepswx37M5NXHRlQiBY-1-pDrAYr00Y4

Downloading...
From: https://drive.google.com/uc?id=1kepswx37M5NXHRlQiBY-1-pDrAYr00Y4
To: /content/cc.fa.300.bin
100% 7.25G/7.25G [01:12<00:00, 99.9MB/s]


In [ ]:
!pip install fasttext -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 68.8/68.8 KB 2.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [ ]:
import fasttext.util

In [ ]:
persian_fasttext_model = fasttext.load_model('/content/cc.fa.300.bin')

In [ ]:
persian_fasttext_model.get_dimension()

300

In [ ]:
vec1 = persian_fasttext_model.get_word_vector('سلام')

In [ ]:
vec1

In [ ]:
persian_fasttext_model.get_word_vector('سلام').shape

(300,)

In [ ]:
text1 ="باید در کارهای خود فکر کنیم	"

In [ ]:
vec2 = persian_fasttext_model.get_word_vector(text1)

In [ ]:
vec2

In [ ]:
vec2.shape

(300,)

In [ ]:
import pandas as pd
import random as rd
import json
import numpy as np
import re

In [ ]:
input = pd.read_csv("/content/input_text.csv")

In [ ]:
input

,Unnamed: 0,input_id,input_text
0,0,0,باید حقوق دیگران را رعایت کنیم
1,1,1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...
2,2,2,ما باید از غیبت کردن پرهیز کنیم
3,3,3,باید خاک وطن را دوست داشت
4,4,4,نباید براساس ظاهر افراد را قضاوت کرد
5,5,5,باید تجارب خود را به افراد دیگر منتقل کنیم
6,6,6,تفاوت قائل شدن بین انسان‌ها درست است
7,7,7,ثروت چیز خوبی است
8,8,8,باید قبل از سخن گفتن فکر کرد.
9,9,9,باید قدردان دارایی‌های خودمان باشیم


In [ ]:
def create_vocabulary(corpus):
    vocabulary = {}
    i = 0
    for s in corpus:
        for w in s.split():
            if w not in vocabulary:
                vocabulary[w] = i
                i+=1
    return vocabulary

In [ ]:
input_vocabulary = create_vocabulary(input['input_text'])

**Calculating the input tf-idf weights**

In [ ]:
# Calculate the TF-IDF weights
from sklearn.feature_extraction.text import TfidfVectorizer
def calculate_tf_idf_weights(text , costum_vocabulary):  
  vectorizer = TfidfVectorizer(vocabulary = costum_vocabulary)
  text_tfidf = vectorizer.fit_transform(text)
  return text_tfidf

In [ ]:
input_tfidf = calculate_tf_idf_weights(input['input_text'], input_vocabulary)

**Calculating the input Embedding**

In [ ]:
def calculate_vector(text , i , vocab):
  sum = 0
  for word in text.split():
    word_index = vocab[word]
    tfidf = input_tfidf[i, word_index]
    w  = persian_fasttext_model.get_word_vector(word)
    sum += w * tfidf
  return sum

In [ ]:
columns = ["input_text","vector","input_id"]
input_table = pd.DataFrame([], columns=columns)
for i in range(len(input)):
  sample =  input.iloc[i]['input_text'] 
  input_table = input_table.append({'input_text':sample ,'vector':[calculate_vector(sample,i, input_vocabulary)],'input_id':input.iloc[i]["input_id"]}, ignore_index = True)

In [ ]:
input_table

,input_text,vector,input_id
0,باید حقوق دیگران را رعایت کنیم,"[[0.0045178575, 0.14212437, -0.094673775, 0.04...",0
1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,"[[-0.050662134, 0.13333057, -0.14714913, -0.08...",1
2,ما باید از غیبت کردن پرهیز کنیم,"[[0.062243972, 0.06898309, -0.06263755, -0.118...",2
3,باید خاک وطن را دوست داشت,"[[0.121969834, 0.07435557, -0.24708495, 0.0969...",3
4,نباید براساس ظاهر افراد را قضاوت کرد,"[[0.033400156, 0.18923937, -0.01732734, -0.009...",4
5,باید تجارب خود را به افراد دیگر منتقل کنیم,"[[-0.05721751, 0.27314007, -0.08946025, -0.054...",5
6,تفاوت قائل شدن بین انسان‌ها درست است,"[[0.2066299, 0.204447, -0.1448164, 0.01504367,...",6
7,ثروت چیز خوبی است,"[[0.041614477, 0.19184005, -0.013416044, -0.04...",7
8,باید قبل از سخن گفتن فکر کرد.,"[[0.18891014, 0.2516171, -0.033496737, -0.0889...",8
9,باید قدردان دارایی‌های خودمان باشیم,"[[0.006277969, 0.14454277, -0.029481124, -0.00...",9


In [ ]:
# pd.DataFrame(input_table).to_csv("/content/input_table.csv")

In [ ]:
def cosine_similarity(list1, list2):    
  # Convert the lists to numpy arrays
  array1 = np.array(list1)
  array2 = np.array(list2)
  # Compute the dot product of the two arrays
  dot_product = np.dot(array1, array2)
  # Compute the magnitude (or length) of each array
  magnitude1 = np.sqrt(np.sum(array1**2))
  magnitude2 = np.sqrt(np.sum(array2**2))
  # Compute the cosine similarity
  cos_sim = dot_product / (magnitude1 * magnitude2)
  return cos_sim

**HAFEZ**

In [ ]:
hafez = pd.read_csv("/content/hafez_normalized.csv")
hafez

,Unnamed: 0,output_id,output_text,qazal_number
0,0,0,الا یا ایها الساقی ادر کاسا و ناولها که عشق آس...,﻿غزل ۱
1,1,1,صلاح کار کجا و من خراب کجا ببین تفاوت ره کز کج...,غزل ۲
2,2,2,اگر آن ترک شیرازی به دست آرد دل ما را به خال ه...,غزل ۳
3,3,3,صبا به لطف بگو آن غزال رعنا را که سر به کوه و ...,غزل ۴
4,4,4,دل می‌رود ز دستم صاحب دلان خدا را دردا که راز ...,غزل ۵
...,...,...,...,...
490,490,490,به چشم کرده‌ام ابروی ماه سیمایی خیال سبزخطی نق...,غزل ۴۹۱
491,491,491,سلامی چو بوی خوش آشنایی بدان مردم دیده روشنایی...,غزل ۴۹۲
492,492,492,ای پادشه خوبان داد از غم تنهایی دل بی تو به جا...,غزل ۴۹۳
493,493,493,ای دل گر از آن چاه زنخدان به درآیی هر جا که رو...,غزل ۴۹۴


In [ ]:
hafez_vocabulary = create_vocabulary(hafez['output_text'])

**Calculate the tf-idf weights for hafez**

In [ ]:
hafez_tfidf = calculate_tf_idf_weights(hafez['output_text'], hafez_vocabulary)

/usr/local/lib/python3.9/dist-packages/sklearn/feature_extraction/text.py:1379: UserWarning: Upper case characters found in vocabulary while 'lowercase' is True. These entries will not be matched with any documents
  warnings.warn(


In [ ]:
hafez['vector'] = np.ndarray

In [ ]:
hafez

,Unnamed: 0,output_id,output_text,qazal_number,vector
0,0,0,الا یا ایها الساقی ادر کاسا و ناولها که عشق آس...,﻿غزل ۱,<class 'numpy.ndarray'>
1,1,1,صلاح کار کجا و من خراب کجا ببین تفاوت ره کز کج...,غزل ۲,<class 'numpy.ndarray'>
2,2,2,اگر آن ترک شیرازی به دست آرد دل ما را به خال ه...,غزل ۳,<class 'numpy.ndarray'>
3,3,3,صبا به لطف بگو آن غزال رعنا را که سر به کوه و ...,غزل ۴,<class 'numpy.ndarray'>
4,4,4,دل می‌رود ز دستم صاحب دلان خدا را دردا که راز ...,غزل ۵,<class 'numpy.ndarray'>
...,...,...,...,...,...
490,490,490,به چشم کرده‌ام ابروی ماه سیمایی خیال سبزخطی نق...,غزل ۴۹۱,<class 'numpy.ndarray'>
491,491,491,سلامی چو بوی خوش آشنایی بدان مردم دیده روشنایی...,غزل ۴۹۲,<class 'numpy.ndarray'>
492,492,492,ای پادشه خوبان داد از غم تنهایی دل بی تو به جا...,غزل ۴۹۳,<class 'numpy.ndarray'>
493,493,493,ای دل گر از آن چاه زنخدان به درآیی هر جا که رو...,غزل ۴۹۴,<class 'numpy.ndarray'>


**Calculating the embedding of the HAFEZ output**

In [ ]:
for sentence_idx in range(len(hafez['output_text'])):
  sum = 0
  for word in hafez['output_text'][sentence_idx].split():
    word_index = hafez_vocabulary[word]
    tfidf = hafez_tfidf[sentence_idx, word_index]
    w = persian_fasttext_model.get_word_vector(word)
    sum += w * tfidf
  hafez['vector'][sentence_idx] = sum

<ipython-input-40-12512785d435>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  hafez['vector'][sentence_idx] = sum


In [ ]:
hafez

,Unnamed: 0,output_id,output_text,qazal_number,vector
0,0,0,الا یا ایها الساقی ادر کاسا و ناولها که عشق آس...,﻿غزل ۱,"[0.16277598, 0.28875795, -0.15890753, -0.30524..."
1,1,1,صلاح کار کجا و من خراب کجا ببین تفاوت ره کز کج...,غزل ۲,"[0.17340647, 2.627466, -1.8306364, 0.18155445,..."
2,2,2,اگر آن ترک شیرازی به دست آرد دل ما را به خال ه...,غزل ۳,"[-0.10190295, 0.5266237, -0.22516805, -0.22914..."
3,3,3,صبا به لطف بگو آن غزال رعنا را که سر به کوه و ...,غزل ۴,"[0.04193136, 0.5171374, -0.015479742, -0.26901..."
4,4,4,دل می‌رود ز دستم صاحب دلان خدا را دردا که راز ...,غزل ۵,"[-0.07424444, 0.5940255, -0.31174916, -0.43841..."
...,...,...,...,...,...
490,490,490,به چشم کرده‌ام ابروی ماه سیمایی خیال سبزخطی نق...,غزل ۴۹۱,"[0.04344524, 0.43918306, 0.0073105707, -0.4758..."
491,491,491,سلامی چو بوی خوش آشنایی بدان مردم دیده روشنایی...,غزل ۴۹۲,"[0.17718472, 0.5374242, -0.3269776, -0.4641924..."
492,492,492,ای پادشه خوبان داد از غم تنهایی دل بی تو به جا...,غزل ۴۹۳,"[0.5083297, 0.77104676, -0.2921334, -0.6966634..."
493,493,493,ای دل گر از آن چاه زنخدان به درآیی هر جا که رو...,غزل ۴۹۴,"[-0.12134411, 0.37779418, -0.33961013, -0.2422..."


In [ ]:
indexes = []
for i in range(len(hafez)):
  if (hafez.iloc[i]["vector"] @ hafez.iloc[i]["vector"].T == 0):
    print(hafez.iloc[i]["vector"])
    print(str(i))
    print(hafez.iloc[i]["output_text"])
    indexes.append(i)

In [ ]:
indexes

[]

In [ ]:
def create_pair_knn(k,model_name,input_table,output_table,isChaptered,chapter_number):
  if isChaptered: 
    columns = ["model","input_id","input_text","output_id","output_text","k","chapter_number","input_vector","output_vector"]
  else:
    columns = ["model","input_id","input_text","output_id","output_text","k","input_vector","output_vector"]

  pair_table = pd.DataFrame([], columns=columns)
  for i in range(len(input_table)):
    vector = []
    for j in range(len(output_table)): 
      input_vector = input_table.iloc[i]['vector']
      input_vector = np.array(input_vector)
      output_vector = output_table.iloc[j]['vector']
      sim = abs(input_vector[0] @ output_vector.T)
      w2_sqrt = np.sqrt(abs(output_vector @ output_vector.T))
      w1_sqrt = np.sqrt(abs(input_vector[0] @ input_vector[0].T))
      
      tmp = sim / (w1_sqrt * w2_sqrt)
      vector.append(tmp)
    vector = np.array(vector)
    indexes = vector.argsort()[-k:]
    p = 1
    for index in indexes:
      if isChaptered: 
        pair_table = pair_table.append({'model':model_name,"input_id":input_table.iloc[i]['input_id'],"input_text":input_table.iloc[i]['input_text'],"output_id":output_table.iloc[index]['output_id'] ,"output_text":output_table.iloc[index]['output_text'] ,"k":p,"chapter_number":output_table.iloc[index][chapter_number],"input_vector":input_table.iloc[i]['vector'][0], "output_vector":output_table.iloc[index]['vector']}, ignore_index = True)
      else:
        pair_table = pair_table.append({'model':model_name,"input_id":input_table.iloc[i]['input_id'],"input_text":input_table.iloc[i]['input_text'],"output_id":output_table.iloc[index]['output_id'] ,"output_text":output_table.iloc[index]['output_text'] ,"k":p,"input_vector":input_table.iloc[i]['vector'], "output_vector":output_table.iloc[index]['vector']}, ignore_index = True)
      p+=1
  return pair_table

**Calculating the hafez-pairs with KNN Algorithm**

In [ ]:
hafez_pair = create_pair_knn(2,"fasttext",input_table,hafez,True,"qazal_number")

In [ ]:
hafez_pair

,model,input_id,input_text,output_id,output_text,k,chapter_number,input_vector,output_vector
0,fasttext,0,باید حقوق دیگران را رعایت کنیم,243,معاشران گره از زلف یار باز کنید شبی خوش است بد...,1,غزل ۲۴۴,"[0.0045178575, 0.14212437, -0.094673775, 0.049...","[0.59435856, 1.0003383, -0.2727954, -0.2040587..."
1,fasttext,0,باید حقوق دیگران را رعایت کنیم,377,ما نگوییم بد و میل به ناحق نکنیم جامه کس سیه و...,2,غزل ۳۷۸,"[0.0045178575, 0.14212437, -0.094673775, 0.049...","[0.15594383, 0.69853926, -0.43618798, 0.335283..."
2,fasttext,1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,53,ز گریه مردم چشمم نشسته در خون است ببین که در ط...,1,غزل ۵۴,"[-0.050662134, 0.13333057, -0.14714913, -0.089...","[0.20546702, 0.7569201, -0.62407655, -0.800692..."
3,fasttext,1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,54,خم زلف تو دام کفر و دین است ز کارستان او یک شم...,2,غزل ۵۵,"[-0.050662134, 0.13333057, -0.14714913, -0.089...","[0.4641765, 0.54866195, -0.36828122, -0.592364..."
4,fasttext,2,ما باید از غیبت کردن پرهیز کنیم,9,دوش از مسجد سوی میخانه آمد پیر ما چیست یاران ط...,1,غزل ۱۰,"[0.062243972, 0.06898309, -0.06263755, -0.1183...","[-0.71047944, -0.98896, 0.043391336, -0.838729..."
...,...,...,...,...,...,...,...,...,...
95,fasttext,47,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,5,به ملازمان سلطان که رساند این دعا را که به شکر...,2,غزل ۶,"[0.20672798, 0.13723634, 0.057638958, -0.01138...","[-0.12489711, 0.31118724, -0.030201748, -0.336..."
96,fasttext,48,باید از منافق دوری کنیم,313,دوش بیماری چشم تو ببرد از دستم لیکن از لطف لبت...,1,غزل ۳۱۴,"[0.042932272, 0.15004778, -0.16598155, 0.05481...","[0.30514216, 0.6867946, -0.35656255, -0.586633..."
97,fasttext,48,باید از منافق دوری کنیم,403,می‌فکن بر صف رندان نظری بهتر از این بر در میکد...,2,غزل ۴۰۴,"[0.042932272, 0.15004778, -0.16598155, 0.05481...","[-0.08670333, 1.4457531, -0.5876103, -0.427993..."
98,fasttext,49,لزوما رسیدن به چیزی که آرزو داریم به نفع ما نیست,490,به چشم کرده‌ام ابروی ماه سیمایی خیال سبزخطی نق...,1,غزل ۴۹۱,"[-0.0021986295, 0.03348967, 0.007216051, -0.08...","[0.04344524, 0.43918306, 0.0073105707, -0.4758..."


**saving the output into the csv file**

In [ ]:
pd.DataFrame(hafez_pair).to_csv("/content/hafez_pair.csv")

**shams**

In [ ]:
shams = pd.read_csv("/content/shams_normalized.csv")
shams

,Unnamed: 0,output_id,output_text,qazal_number
0,0,495,ای رستخیز ناگهان، وی رحمت بی منتها ای آتشی افر...,1
1,1,496,ای یوسف خوش نام ما خوش می روی بر بام ما ای درش...,4
2,2,497,بگریز ای میر اجل از ننگ ما از ننگ ما زیرا نمی ...,6
3,3,498,جز وی چه باشد کز اجل اندررباید کل ما صد جان بر...,8
4,4,499,من از کجا پند از کجا باده بگردان ساقیا آن جام ...,9
...,...,...,...,...
2373,2373,2868,یا ملک المحشر، ترحم لا ترتشی کل سقیط ردی ترحمه...,3225
2374,2374,2869,قلت له مصیحا یا ملک المشرق اقسم بالخالق مثلک ل...,3226
2375,2375,2870,یا ساقی الراح خذ و امرلائ به طاسی فلست املک صب...,3227
2376,2376,2871,ایا ملتقی العیش کم تبعدی و یا فرقه الحسب کم تع...,3228


In [ ]:
shams_vocabulary = create_vocabulary(shams['output_text'])

In [ ]:
shams_tfidf = calculate_tf_idf_weights(shams['output_text'], shams_vocabulary)

In [ ]:
shams['vector'] = np.ndarray

In [ ]:
shams

,Unnamed: 0,output_id,output_text,qazal_number,vector
0,0,495,ای رستخیز ناگهان، وی رحمت بی منتها ای آتشی افر...,1,<class 'numpy.ndarray'>
1,1,496,ای یوسف خوش نام ما خوش می روی بر بام ما ای درش...,4,<class 'numpy.ndarray'>
2,2,497,بگریز ای میر اجل از ننگ ما از ننگ ما زیرا نمی ...,6,<class 'numpy.ndarray'>
3,3,498,جز وی چه باشد کز اجل اندررباید کل ما صد جان بر...,8,<class 'numpy.ndarray'>
4,4,499,من از کجا پند از کجا باده بگردان ساقیا آن جام ...,9,<class 'numpy.ndarray'>
...,...,...,...,...,...
2373,2373,2868,یا ملک المحشر، ترحم لا ترتشی کل سقیط ردی ترحمه...,3225,<class 'numpy.ndarray'>
2374,2374,2869,قلت له مصیحا یا ملک المشرق اقسم بالخالق مثلک ل...,3226,<class 'numpy.ndarray'>
2375,2375,2870,یا ساقی الراح خذ و امرلائ به طاسی فلست املک صب...,3227,<class 'numpy.ndarray'>
2376,2376,2871,ایا ملتقی العیش کم تبعدی و یا فرقه الحسب کم تع...,3228,<class 'numpy.ndarray'>


**Calculating tf-idf for shams**

In [ ]:
for sentence_idx in range(len(shams['output_text'])):
  sum = 0
  for word in shams['output_text'][sentence_idx].split():
    word_index = shams_vocabulary[word]
    tfidf = shams_tfidf[sentence_idx, word_index]
    w = persian_fasttext_model.get_word_vector(word)
    sum += w * tfidf
  shams['vector'][sentence_idx] = sum

<ipython-input-107-fa251dc964fb>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shams['vector'][sentence_idx] = sum


In [ ]:
indexes = []
for i in range(len(shams)):
  if (shams.iloc[i]["vector"] @ shams.iloc[i]["vector"].T == 0):
    print(shams.iloc[i]["vector"])
    print(str(i))
    print(shams.iloc[i]["output_text"])
    indexes.append(i)

In [ ]:
indexes

[]

In [ ]:
shams

,Unnamed: 0,output_id,output_text,qazal_number,vector
0,0,495,ای رستخیز ناگهان، وی رحمت بی منتها ای آتشی افر...,1,"[0.31516948, 0.43997836, -0.4099239, -0.475878..."
1,1,496,ای یوسف خوش نام ما خوش می روی بر بام ما ای درش...,4,"[-2.6470532, -3.9393294, 0.851688, -1.3556036,..."
2,2,497,بگریز ای میر اجل از ننگ ما از ننگ ما زیرا نمی ...,6,"[-1.6573735, -3.267869, 0.1068527, -1.2031579,..."
3,3,498,جز وی چه باشد کز اجل اندررباید کل ما صد جان بر...,8,"[0.08342255, 0.6493032, -0.57422936, -0.958621..."
4,4,499,من از کجا پند از کجا باده بگردان ساقیا آن جام ...,9,"[0.10159991, 0.7147436, -0.5457657, -0.3151351..."
...,...,...,...,...,...
2373,2373,2868,یا ملک المحشر، ترحم لا ترتشی کل سقیط ردی ترحمه...,3225,"[0.048952974, 0.2859299, -0.3387595, -0.589543..."
2374,2374,2869,قلت له مصیحا یا ملک المشرق اقسم بالخالق مثلک ل...,3226,"[0.19978821, 0.0106124785, -0.14959618, -0.210..."
2375,2375,2870,یا ساقی الراح خذ و امرلائ به طاسی فلست املک صب...,3227,"[0.09569981, 0.16479793, -0.15692888, 0.009008..."
2376,2376,2871,ایا ملتقی العیش کم تبعدی و یا فرقه الحسب کم تع...,3228,"[-0.33792853, 0.121178254, -0.59539163, -0.388..."


**Calculating the pairs with KNN Algorithm**

In [ ]:
shams_pair = create_pair_knn(2,"fasttext",input_table,shams,True,"qazal_number")

In [ ]:
shams_pair

,model,input_id,input_text,output_id,output_text,k,chapter_number,input_vector,output_vector
0,fasttext,0,باید حقوق دیگران را رعایت کنیم,1784,اگر زمین و فلک را پر از سلام کنیم وگر سگان تو ...,1,1731,"[0.0045178575, 0.14212437, -0.094673775, 0.049...","[0.16253407, 1.8121697, -0.7413508, 0.23071525..."
1,fasttext,0,باید حقوق دیگران را رعایت کنیم,1514,آمد بهار ای دوستان منزل سوی بستان کنیم گرد غری...,2,1370,"[0.0045178575, 0.14212437, -0.094673775, 0.049...","[0.26517013, 2.0186355, -0.73137516, 0.3541532..."
2,fasttext,1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,1842,کافرم ار در دو جهان عشق بود خوشتر از این دیده ...,1,1819,"[-0.050662134, 0.13333057, -0.14714913, -0.089...","[0.57857317, 1.0385668, -0.7664281, -0.3210810..."
3,fasttext,1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,2700,اندر میان جمع چه جان است آن یکی یک جان نخوانمش...,2,2995,"[-0.050662134, 0.13333057, -0.14714913, -0.089...","[-0.7407535, 2.4354486, -1.5639377, -0.6870707..."
4,fasttext,2,ما باید از غیبت کردن پرهیز کنیم,1772,ما در جهان موافقت کس نمی کنیم ما خانه زیر گنبد...,1,1712,"[0.062243972, 0.06898309, -0.06263755, -0.1183...","[-0.10094829, 1.5016159, -0.802642, -0.2043204..."
...,...,...,...,...,...,...,...,...,...
95,fasttext,47,هرکاری را می‌توان انجام داد به جز تغییر عادت‌ها,1061,صنما جفا رها کن کرم این روا ندارد بنگر به سوی ...,2,767,"[0.20672798, 0.13723634, 0.057638958, -0.01138...","[0.31134415, 0.60699767, -0.34680873, -0.54822..."
96,fasttext,48,باید از منافق دوری کنیم,1692,بار دیگر از دل و از عقل و جان برخاستیم یار آمد...,1,1601,"[0.042932272, 0.15004778, -0.16598155, 0.05481...","[0.01365528, 0.5440097, -0.6927948, -0.1640346..."
97,fasttext,48,باید از منافق دوری کنیم,1842,کافرم ار در دو جهان عشق بود خوشتر از این دیده ...,2,1819,"[0.042932272, 0.15004778, -0.16598155, 0.05481...","[0.57857317, 1.0385668, -0.7664281, -0.3210810..."
98,fasttext,49,لزوما رسیدن به چیزی که آرزو داریم به نفع ما نیست,2587,هله عاشقان بشارت که نماند این جدایی برسد وصال ...,1,2837,"[-0.0021986295, 0.03348967, 0.007216051, -0.08...","[0.34967864, 0.8905476, -0.29779592, -0.663238..."


In [ ]:
pd.DataFrame(shams_pair).to_csv("/content/shams_pair.csv")


**poem pair**

In [ ]:
poem = pd.read_csv("/content/poem_normalized.csv")
poem

,Unnamed: 0,output_id,output_text
0,0,2873,در سخن گفتن خطای جاهلان پیدا شود تیر کج چون از...
1,1,2874,ترا دیدم و یوسف را شنیدم شنیدن کی بود مانند دیدن
2,2,2875,رزق را روزی رسان پر میدهد بی مگس هرگز نماند عن...
3,3,2876,دل ز پر گفتن بمیرد در بدن گر چه گفتارش بود در عدن
4,4,2877,امشب تو را به خوبی نسبت به ماه کردم تو خوب تر ...
...,...,...,...
345,345,3218,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...
346,346,3219,در جوانی پاک بودن شیوه پیغمبریست ورنه هر گبری ...
347,347,3220,درست نیست دو رنگی میان ظاهر و باطن بگو شکسته ن...
348,348,3221,در غربت مرگ بیم تنهایی نیست یاران عزیز آن طرف ...


In [ ]:
poem_vocabulary = create_vocabulary(poem['output_text'])

**Calculating tf-idf weights for the poem input**

In [ ]:
poem_tfidf = calculate_tf_idf_weights(poem['output_text'], poem_vocabulary)

In [ ]:
poem['vector'] = np.ndarray

In [ ]:
poem

,Unnamed: 0,output_id,output_text,vector
0,0,2873,در سخن گفتن خطای جاهلان پیدا شود تیر کج چون از...,<class 'numpy.ndarray'>
1,1,2874,ترا دیدم و یوسف را شنیدم شنیدن کی بود مانند دیدن,<class 'numpy.ndarray'>
2,2,2875,رزق را روزی رسان پر میدهد بی مگس هرگز نماند عن...,<class 'numpy.ndarray'>
3,3,2876,دل ز پر گفتن بمیرد در بدن گر چه گفتارش بود در عدن,<class 'numpy.ndarray'>
4,4,2877,امشب تو را به خوبی نسبت به ماه کردم تو خوب تر ...,<class 'numpy.ndarray'>
...,...,...,...,...
345,345,3218,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...,<class 'numpy.ndarray'>
346,346,3219,در جوانی پاک بودن شیوه پیغمبریست ورنه هر گبری ...,<class 'numpy.ndarray'>
347,347,3220,درست نیست دو رنگی میان ظاهر و باطن بگو شکسته ن...,<class 'numpy.ndarray'>
348,348,3221,در غربت مرگ بیم تنهایی نیست یاران عزیز آن طرف ...,<class 'numpy.ndarray'>


**Calculating fasttext Embedding for the poem with td-idf weights**

In [ ]:
for sentence_idx in range(len(poem['output_text'])):
  sum = 0
  for word in poem['output_text'][sentence_idx].split():
    word_index = poem_vocabulary[word]
    tfidf = poem_tfidf[sentence_idx, word_index]
    w = persian_fasttext_model.get_word_vector(word)
    sum += w * tfidf
  poem['vector'][sentence_idx] = sum

<ipython-input-123-7f5019cec777>:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  poem['vector'][sentence_idx] = sum


In [ ]:
indexes = []
for i in range(len(shams)):
  if (shams.iloc[i]["vector"] @ shams.iloc[i]["vector"].T == 0):
    print(shams.iloc[i]["vector"])
    print(str(i))
    print(shams.iloc[i]["output_text"])
    indexes.append(i)

In [ ]:
indexes

[]

In [ ]:
poem

,Unnamed: 0,output_id,output_text,vector
0,0,2873,در سخن گفتن خطای جاهلان پیدا شود تیر کج چون از...,"[0.2892229, 0.45664594, -0.17676038, -0.200403..."
1,1,2874,ترا دیدم و یوسف را شنیدم شنیدن کی بود مانند دیدن,"[0.0564308, 0.3019183, -0.16309911, -0.1365846..."
2,2,2875,رزق را روزی رسان پر میدهد بی مگس هرگز نماند عن...,"[0.16360337, 0.26804096, 0.06337275, -0.124764..."
3,3,2876,دل ز پر گفتن بمیرد در بدن گر چه گفتارش بود در عدن,"[0.00054260716, 0.13817486, -0.016994253, -0.1..."
4,4,2877,امشب تو را به خوبی نسبت به ماه کردم تو خوب تر ...,"[0.010124502, 0.20972925, -0.09504011, -0.2131..."
...,...,...,...,...
345,345,3218,در این عالم از هرچه بایستنیست نکوتر ز فرزند شا...,"[0.047969628, 0.217871, -0.04454742, -0.054346..."
346,346,3219,در جوانی پاک بودن شیوه پیغمبریست ورنه هر گبری ...,"[0.19383046, 0.087484956, -0.112393364, -0.103..."
347,347,3220,درست نیست دو رنگی میان ظاهر و باطن بگو شکسته ن...,"[0.059897475, 0.12911692, -0.050175577, -0.196..."
348,348,3221,در غربت مرگ بیم تنهایی نیست یاران عزیز آن طرف ...,"[0.04081338, 0.19527285, -0.1691353, -0.019216..."


**Calculating the pairs with KNN Algorithm**

In [ ]:
poem_pair = create_pair_knn(3,"fasttext", input_table, poem, False, "none")

In [ ]:
poem_pair

,model,input_id,input_text,output_id,output_text,k,input_vector,output_vector
0,fasttext,0,باید حقوق دیگران را رعایت کنیم,3071,موی سپید را فلکم رایگان نداد این رشته را به نق...,1,"[[0.0045178575, 0.14212437, -0.094673775, 0.04...","[0.12667055, 0.22411615, -0.039572157, -0.0361..."
1,fasttext,0,باید حقوق دیگران را رعایت کنیم,3134,بلا ندیده دعا را شروع باید کرد علاج واقعه پیش ...,2,"[[0.0045178575, 0.14212437, -0.094673775, 0.04...","[0.4213421, 0.4327496, -0.18507965, -0.0969296..."
2,fasttext,0,باید حقوق دیگران را رعایت کنیم,2934,آن را که جفا جوست نمی باید خواست سنگین دل و بد...,3,"[[0.0045178575, 0.14212437, -0.094673775, 0.04...","[-0.067745954, 0.37336424, -0.049164154, -0.09..."
3,fasttext,1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,3077,تواضع گر چه محبوب است و فضل بیکران دارد نباید ...,1,"[[-0.050662134, 0.13333057, -0.14714913, -0.08...","[0.049595244, 0.15655851, -0.15584348, -0.2543..."
4,fasttext,1,ادب یکی از صفات اخلاقی نیکو است که انسان باید ...,2924,تواضع گر چه محبوب است و فضل بیکران دارد نباید ...,2,"[[-0.050662134, 0.13333057, -0.14714913, -0.08...","[0.049595244, 0.15655851, -0.15584348, -0.2543..."
...,...,...,...,...,...,...,...,...
145,fasttext,48,باید از منافق دوری کنیم,3118,از جمله رفتگان این راه دراز باز آمده کیست تا ب...,2,"[[0.042932272, 0.15004778, -0.16598155, 0.0548...","[0.19797295, 0.25321278, -0.17183124, -0.12633..."
146,fasttext,48,باید از منافق دوری کنیم,3068,گفتار زیاد نه از نغزیست ولوله طبل ز بی‌مغزیستم...,3,"[[0.042932272, 0.15004778, -0.16598155, 0.0548...","[0.018575028, 0.1848239, -0.025093246, -0.1594..."
147,fasttext,49,لزوما رسیدن به چیزی که آرزو داریم به نفع ما نیست,2973,رسد آدمی به جایی که به جز خدا نبیند بنگر که تا...,1,"[[-0.0021986295, 0.03348967, 0.007216051, -0.0...","[-0.021311985, 0.030127468, -0.043376606, -0.1..."
148,fasttext,49,لزوما رسیدن به چیزی که آرزو داریم به نفع ما نیست,3156,جان رفت و ما به آرزوی دل نمی‌رسیم هرچند میرویم...,2,"[[-0.0021986295, 0.03348967, 0.007216051, -0.0...","[0.033563126, 0.13460839, -0.028407479, -0.062..."


**saving the data into a csv file**

In [ ]:
pd.DataFrame(poem_pair).to_csv("/content/poem_pair.csv")